In [1]:
import pandas as pd

### in data_set  label 1 = fake(unreliable)  , 0 = true(reliable)

In [2]:
train_df = pd.read_csv("train.csv")
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
train_df.shape


(20800, 5)

In [4]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [5]:
test_df.shape

(5200, 4)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
train_df = train_df.dropna()
train_df.shape

(18285, 5)

In [8]:
x = train_df.drop('label' , axis = 1)
y = train_df['label']
x.shape  ,  y.shape

((18285, 4), (18285,))

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [10]:
### Vocabulary size 
voc_size = 7000

In [130]:
# Onehot Representation
messages = x.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [12]:
messages.reset_index(inplace = True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [14]:
### Data cleaning 

ps = PorterStemmer()
clean_Data = []
for i in range(0, len(messages)):
#     print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    clean_Data.append(review)


In [15]:
clean_Data[1:10]

['flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [16]:
# convert in one hot encoding 

onehot_rep = [one_hot(word,voc_size) for word in clean_Data]
onehot_rep[1:10]

[[115, 5647, 4192, 1186, 3, 2976, 6472],
 [6171, 2833, 2670, 6239],
 [2773, 6957, 3417, 3238, 2353, 3424],
 [94, 3, 3158, 5404, 2761, 1419, 3, 3758, 6052, 5963],
 [3417,
  2566,
  1689,
  6143,
  6845,
  4524,
  641,
  3593,
  1486,
  2481,
  286,
  5288,
  3660,
  1564,
  6472],
 [3535, 5759, 816, 121, 5687, 1829, 6079, 447, 1374, 1569, 1739],
 [6791, 286, 1006, 6228, 3445, 4693, 4524, 4347, 1374, 1569, 1739],
 [5907, 3803, 1517, 1620, 4775, 3445, 3616, 1672, 4524, 4313],
 [1666, 5268, 2473, 6190, 6408, 4405, 2577, 5837]]

In [17]:
#  Embeding representations

# pad_sequence 
sent_length = 22
embedded_docs = pad_sequences(onehot_rep , padding = 'pre' , maxlen=sent_length )

print(embedded_docs)

[[   0    0    0 ... 3900 1527 4403]
 [   0    0    0 ...    3 2976 6472]
 [   0    0    0 ... 2833 2670 6239]
 ...
 [   0    0    0 ... 1374 1569 1739]
 [   0    0    0 ...  674 4647 4723]
 [   0    0    0 ... 6690   76 5439]]


### input_dim : Size of the vocabulary
### output_dim : Length of the vector for each word
### input_length : Maximum length of a sequence

In [ ]:
#  creating model
embeding_vector_feature = 40 
model = Sequential()
model.add(Embedding(voc_size , embeding_vector_feature , input_length=sent_length))
model.add(LSTM(100))    #m  100 is number of nerons
model.add(Dense(1 , activation='sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer='adam' , metrics=['accuracy'])
model.summary()

In [19]:
len(embedded_docs),y.shape


(18285, (18285,))

In [20]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)


In [21]:
X_final.shape,y_final.shape


((18285, 22), (18285,))

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [23]:
model.fit(X_train , y_train , validation_data=(X_test , y_test) , epochs = 10 ,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 44ms/step - loss: 0.3312 - accuracy: 0.8463 - val_loss: 0.1943 - val_accuracy: 0.9191
Epoch 2/10
192/192 [==============================] - 8s 40ms/step - loss: 0.1310 - accuracy: 0.9492 - val_loss: 0.1938 - val_accuracy: 0.9181
Epoch 3/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0871 - accuracy: 0.9682 - val_loss: 0.2215 - val_accuracy: 0.9200
Epoch 4/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0593 - accuracy: 0.9794 - val_loss: 0.2466 - val_accuracy: 0.9163
Epoch 5/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0428 - accuracy: 0.9863 - val_loss: 0.3180 - val_accuracy: 0.9133
Epoch 6/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0330 - accuracy: 0.9900 - val_loss: 0.2906 - val_accuracy: 0.9123
Epoch 7/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0214 - accuracy: 0.9939 - val_loss: 0.4023 - val_accuracy: 0.914

In [67]:
y_pred=model.predict(X_test)
y_pred = y_pred.round()

In [68]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)


array([[3086,  333],
       [ 190, 2426]], dtype=int64)

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.9133388566694284

## Create function that Predict the sentance that is reliable or not 

In [120]:
def predict_sentance(str1):
#     clean the sentance
    review = re.sub('[^a-zA-Z]', ' ', str1)
    review = review.lower()
    review = review.split()   
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    # one_hot representation 
    t1 = [one_hot(review , voc_size)]
    # # pad_sequences 
    t1_doc = pad_sequences(t1 , padding = 'pre' , maxlen = 22)
    predict = model.predict(t1_doc).round()
    print(predict)
    if predict == 1.0 :
        print("Unreliable " , predict)
    else:
        print("Reliable " , predict)

In [123]:
t1 = 'Mosquito Army Released in Zika Fight in Brazil & Colombia'
predict_sentance(t1)

[[1.]]
Unreliable  [[1.]]


In [129]:
t2 = 'Specter of Trump Loosens Tongues, if Not Purse Strings, in Silicon Valley - The New York Times'
predict_sentance(t2)

[[0.]]
Reliable  [[0.]]
